In [2]:
import pandas as pd
predictions = pd.read_csv('../data/processed/predictions.csv')
optimal_prices = pd.read_csv('../data/processed/optimal_prices.csv')

In [3]:
import matplotlib.pyplot as plt
# Plot the demand curve for this SKU
sku_results = predictions[predictions['SKU'] == sku]
plt.plot(sku_results['price'], sku_results['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
plt.xlabel('Price')
plt.ylabel('Predicted Sales')
plt.legend()

    # Show the plot
plt.show()

NameError: name 'sku' is not defined

In [ ]:
import matplotlib.pyplot as plt

def perform_inference(train_data, test_data):
    # Fit the XGBoost model
    xgboost_model = XGBRegressor()
    xgboost_model.fit(train_data.drop(columns=['SKU', 'sales']), train_data['sales'])

    # Group test data by SKU
    grouped = test_data.groupby('SKU')

    # Create an empty dataframe to store the results
    results = pd.DataFrame(columns=['SKU', 'price', 'predicted_sales_xgboost'])

    # Perform inference for each SKU in test data
    for sku, group_data in grouped:
        print(f"Performing inference for SKU: {sku}")

        # Select one row from group data (one set of feature values for this SKU)
        feature_values = group_data.iloc[0]

        # Set the base price from data
        base_price = feature_values['price']

        # Define the price points
        price_points = [base_price, 
                        base_price * 0.9,   # base_price - 10%
                        base_price * 0.85,  # base_price - 15%
                        base_price * 1.1,   # base_price + 10%
                        base_price * 1.15]  # base_price + 15%

        # For each price point, use the trained model to predict sales and print the result
        for price in price_points:
            # Change the price while keeping other features constant
            test_features = feature_values.copy()
            test_features['price'] = price

            # Remove the SKU field to match the input shape expected by the model
            test_features = test_features.drop(['SKU', 'sales'])
 
            # Predict sales using the XGBoost model and append the results to the dataframe
            predicted_sales_xgboost = xgboost_model.predict(test_features.values.reshape(1, -1))[0]
            results = results.append({'SKU': sku, 'price': price, 'predicted_sales_xgboost': predicted_sales_xgboost}, ignore_index=True)

        # Plot the demand curve for this SKU
        sku_results = results[results['SKU'] == sku]
        plt.plot(sku_results['price'], sku_results['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
    plt.xlabel('Price')
    plt.ylabel('Predicted Sales')
    plt.legend()

    # Show the plot
    plt.show()

    return results

In [ ]:
import matplotlib.pyplot as plt

def plot_demand_curve(predictions, sku):
    # Select the predictions for the given SKU
    sku_predictions = predictions[predictions['SKU'] == sku]

    # Plot the predicted sales against the price points
    plt.plot(sku_predictions['price'], sku_predictions['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
    plt.xlabel('Price')
    plt.ylabel('Predicted Sales')
    plt.legend()

    # Show the plot
    plt.show()

In [11]:
# load datasets
import pandas as pd
predictions = pd.read_csv("../data/processed/predictions.csv")
optimal_prices = pd.read_csv("../data/processed/optimal_prices.csv")
amazon = pd.read_csv("../data/raw/Amazon_Sale_Report.csv")
pl = pd.read_csv("../data/raw/PLMarch2021.csv")

predictions = predictions.merge(amazon[['SKU', 'Category']], on='SKU', how='left')

/var/folders/qd/2qwtnqg57r915dp6p3dq762c0000gn/T/ipykernel_35450/2272987174.py:5: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon = pd.read_csv("../data/raw/Amazon_Sale_Report.csv")


In [5]:
# rename sku column to match optimal prices
pl.rename(columns={'Sku': 'SKU'}, inplace=True)

In [6]:
# write and an underscore on whitespace for every column in pl dataframe
pl.columns = pl.columns.str.replace(' ', '_')

In [7]:

pl.Category.rename({'Kurta': 'kurta', 'Kurta Set':'kurta', 'Gown':'Ethnic Dress', 'Tops':'Set','Nill':'Top' }, inplace=True)




In [8]:
#print(pl.head().to_string(index=False))
competitor_columns = ['Amazon_MRP', 'Flipkart_MRP', 'Ajio_MRP', 'Limeroad_MRP', 'Myntra_MRP', 'Paytm_MRP', 'Snapdeal_MRP']
from src.data.utils import Utils
# convert to int the competitor_columns in the pl dataframe
for col in competitor_columns:
    pl[col] = pl[col].apply(Utils.convert_to_int)
    

#avg_prices = [pl[col].mean() for col in competitor_columns]

In [9]:
avg_prices = [pl[col].mean() for col in competitor_columns]

In [10]:

import gradio as gr

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import plotly.graph_objs as go



    
unique_skus = sorted(predictions['SKU'].unique())
unique_category = sorted(pl['Category'].unique())
unique_price_points = sorted(predictions['price'].unique())
min_price_points = int(min(sorted(predictions["price"].unique())))
max_price_points = int(max(sorted(predictions["price"].unique())))

def plot_demand_curve_gradio(sku, price_points):
    # Load the predictions dataframe
    #predictions = pd.read_csv('../data/processed/predictions.csv')

    # Select the predictions for the given SKU
    sku_predictions = predictions[predictions['SKU'] == sku]

    # Filter the predictions based on price_points (only showing predictions where the price is less than or equal to price_points
    sku_predictions = sku_predictions[sku_predictions['price'] <= price_points]


    # Create a scatter plot of the predicted sales against the price points
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sku_predictions['price'], y=sku_predictions['predicted_sales_xgboost'], mode='lines+markers'))


    # Set the plot title and axis labels
    fig.update_layout(title=f'Demand Curve for SKU {sku}', xaxis_title='Price', yaxis_title='Predicted Sales')
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the legend font size and position
    fig.update_layout(legend=dict(font=dict(size=14), yanchor='top', y=0.99, xanchor='left', x=0.01), legend_title_text='SKU')

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))


    # Return the plot as an HTML div
    return gr.update(value=fig, visible=True)

def plot_optimal_prices_gradio(sku):
    # Load the optimal prices dataframe
    #optimal_prices = pd.read_csv('../data/processed/optimal_prices.csv')

    # Select the optimal prices for the given SKU
    sku_optimal_prices = optimal_prices[optimal_prices['SKU'] == sku]

    # Create a bar chart of the optimal prices at different price points
    fig = go.Figure()
    fig.add_trace(go.Bar(x=['Base Price', '-10%', '-15%', '+10%', '+15%'], y=sku_optimal_prices['optimal_prices'], name=sku))

    # Set the plot title and axis labels
    fig.update_layout(title=f'Optimal Prices for SKU {sku}', xaxis_title='Price Point', yaxis_title='Optimal Price', font=dict(size=14))
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))

    # Return the plot as an HTML div
    return gr.update(value=fig, visible=True)

def plot_competitors_prices_gradio(category):

    sku_competitors_results = pl[pl['Category'] == category]


    # Calculate average prices for each competitor within the selected category
    competitor_columns = ['Amazon_MRP', 'Flipkart_MRP', 'Ajio_MRP', 'Limeroad_MRP', 'Myntra_MRP', 'Paytm_MRP', 'Snapdeal_MRP']
    avg_prices = [sku_competitors_results[col].median() for col in competitor_columns]

    # Create a bar chart of the average competitors prices
    fig = go.Figure()
    fig.add_trace(go.Bar(x=competitor_columns, y=avg_prices, name=category))

    # set the plot title and axis labels
    fig.update_layout(title=f'Competitors Prices for Category {category}', xaxis_title='Competitor', yaxis_title='Median Price', font=dict(size=14))
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))

    # return the plot as an HTML div
    return gr.update(value=fig, visible=True)




with gr.Blocks() as demo:
    gr.Markdown("""
    # Relu Demand Curve & Optimal Pricing Dashboard 💰 #
    
   Version: 1.0.0
    
    Enter Values below then click Predict and Pricing Buttons to see results.  
    
    """)
    with gr.Row(equal_height=True):
      
        sku = gr.Dropdown(
            label="Select SKU",
            choices=unique_skus,
            value=lambda: random.choice(unique_skus),
        )

        price_points = gr.Slider(label="Price points", minimum=min_price_points, maximum=max_price_points, step=1)

    with gr.Row(equal_height=True):

        with gr.Column():
            plot = gr.Plot()
            with gr.Row():
                predict_btn = gr.Button(value="Predict")
            predict_btn.click(
                plot_demand_curve_gradio,
                inputs=[sku, price_points],

                outputs= plot

            )
        with gr.Column():

            plot = gr.Plot()
            with gr.Row():

                optimal_price_btn = gr.Button(value="Optimal Price")
                optimal_price_btn.click(
                    plot_optimal_prices_gradio,
                    inputs=[sku],
                    outputs=plot

                )
        # return plot_comeptitors_prices_gradio(sku) as a plot for every sku
    with gr.Row(equal_height=True):
            category = gr.Dropdown(
            label="Select Category",
            choices=unique_category,
            value=lambda: random.choice(unique_category))


            with gr.Column():
                plot = gr.Plot()
                with gr.Row():
                    competitor_btn = gr.Button(value="Competitors Prices")
                    competitor_btn.click(
                        plot_competitors_prices_gradio,
                        inputs=[category],
                        outputs=plot)



demo.launch(share=False)



/Users/servandodavidtorresgarcia/miniconda3/envs/relu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Create fake data
date = pd.date_range(start='01-01-2022', periods=100)
price_sku1 = np.random.randint(low=10, high=20, size=100)
price_sku2 = np.random.randint(low=15, high=25, size=100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=date, y=price_sku1, mode='lines', name='SKU1'))
fig.add_trace(go.Scatter(x=date, y=price_sku2, mode='lines', name='SKU2'))
fig.update_layout(title='SKU Pricing Trend Over Time', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [2]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=price_sku1, name='SKU1'))
fig.add_trace(go.Histogram(x=price_sku2, name='SKU2'))
fig.update_layout(barmode='overlay', title='SKU Price Distribution', xaxis_title='Price', yaxis_title='Frequency')
fig.update_traces(opacity=0.75)
fig.show()


In [8]:
competitor_price_sku1 = np.random.randint(low=12, high=22, size=100)
competitor_price_sku2 = np.random.randint(low=17, high=27, size=100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=date, y=price_sku1, mode='lines', name='Our SKU1'))
fig.add_trace(go.Scatter(x=date, y=competitor_price_sku1, mode='lines', name='Competitor SKU1'))
fig.update_layout(title='SKU Price Comparison with Competitors', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [4]:
quantity_sku1 = 200 - 5*price_sku1 + np.random.normal(0, 10, 100)
quantity_sku2 = 200 - 4*price_sku2 + np.random.normal(0, 10, 100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=price_sku1, y=quantity_sku1, mode='markers', name='SKU1'))
fig.add_trace(go.Scatter(x=price_sku2, y=quantity_sku2, mode='markers', name='SKU2'))
fig.update_layout(title='Price Elasticity of SKUs', xaxis_title='Price', yaxis_title='Quantity Sold')
fig.show()


In [5]:
# Calculate percentage price change
price_change_sku1 = np.diff(price_sku1) / price_sku1[:-1] * 100
price_change_sku2 = np.diff(price_sku2) / price_sku2[:-1] * 100

fig = go.Figure()
fig.add_trace(go.Bar(x=date[1:], y=price_change_sku1, name='SKU1'))
fig.add_trace(go.Bar(x=date[1:], y=price_change_sku2, name='SKU2'))
fig.update_layout(barmode='relative', title='Percentage Price Change of SKUs Over Time', xaxis_title='Date', yaxis_title='Percentage Price Change')
fig.show()


In [2]:
import plotly.express as px
import pandas as pd

# create a sample dataframe with product information
df = pd.DataFrame({
    'product': ['A', 'B', 'C', 'D', 'E'],
    'cost': [10, 15, 20, 25, 30],
    'sell_price': [5, 10, 15, 20, 25]
})

# create a new column to calculate the profit margin
df['profit_margin'] = (df['sell_price'] - df['cost']) / df['cost']

# create a scatter plot to represent products where the sell prices are lower than the cost
fig = px.scatter(df[df['profit_margin'] < 0], x='product', y='profit_margin', color='product')

# set the plot title and axis labels
fig.update_layout(title='Products with Negative Profit Margin', xaxis_title='Product', yaxis_title='Profit Margin')

# add annotations to provide more context and explanation
fig.add_annotation(x=0.5, y=-0.2, text='This product is being sold at a loss', showarrow=False)
fig.add_annotation(x=1.5, y=-0.1, text='This product is barely breaking even', showarrow=False)
fig.add_annotation(x=2.5, y=0.0, text='This product is making a small profit', showarrow=False)
fig.add_annotation(x=3.5, y=0.1, text='This product is making a good profit', showarrow=False)
fig.add_annotation(x=4.5, y=0.2, text='This product is making a great profit', showarrow=False)

# show the plot
fig.show()